In [1]:
import matplotlib.pyplot as plt
import jax.numpy as jnp
from rlllm_utils import MAB, ThompsonSampler, LLMAgent

In [2]:
models = [
    'gemini-2.5-flash-preview-04-17',
    'gemini-2.5-pro-preview-05-06'
]

temperatures = jnp.linspace(0, 1, 11)

instructions = {
    'vanilla': """Given the history of actions and rewards, you must choose the next action, an int in [0-n_arms-1]. Note, you should return only the next action, nothing else.
    """,
    'rl-prompted': ""
}

DEBUG:jax._src.dispatch:Finished tracing + transforming subtract for pjit in 0.0006837844848632812 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming true_divide for pjit in 0.0004162788391113281 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming true_divide for pjit in 0.00036334991455078125 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming subtract for pjit in 0.0005459785461425781 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming multiply for pjit in 0.000255584716796875 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming add for pjit in 0.0003783702850341797 sec
DEBUG:jax._src.dispatch:Finished tracing + transforming _linspace for pjit in 0.010624408721923828 sec
DEBUG:jax._src.xla_bridge:No jax_plugins namespace packages available
DEBUG:jax._src.xla_bridge:Initializing backend 'cpu'
DEBUG:jax._src.xla_bridge:Backend 'cpu' initialized
DEBUG:jax._src.xla_bridge:Initializing backend 'cuda'
INFO:jax._src.xla_bridge:Unable to initialize bac

In [3]:
# --- Example Usage & Experiment Setup ---
def run_experiment(mab_env, agent, N, initial_observations):
    for i, (arm, reward) in enumerate(initial_observations):
        # print(f"Observation {i+1}: Pulled Arm {arm}, Got Reward {reward}")
        agent.update(arm, reward)

    policy_samples = [agent.select_arm() for _ in range(N)]
    action_distribution = {i: policy_samples.count(i) / N for i in range(mab_env.n_arms)}
    return action_distribution

In [5]:

# --- Configuration ---
ARM_PROBABILITIES = [0.2, 0.5, 0.8] # True probabilities of reward for each arm
N_ARMS = len(ARM_PROBABILITIES)

# Fixed set of initial observations O
# (arm_index, reward_received)
INITIAL_OBSERVATIONS_O = [
    (0, 0), (1, 1), (0, 0), (2, 1), (1, 0), (2, 1)
]
# For example:
# Arm 0 (prob 0.2) was pulled twice, got 0 reward both times.
# Arm 1 (prob 0.5) was pulled twice, got 1 reward once, 0 reward once.
# Arm 2 (prob 0.8) was pulled twice, got 1 reward both times.

LLM_TEMPERATURE = 0.7 # Temperature for LLM generation
N = 100

# --- Setup ---
mab = MAB(arm_probabilities=ARM_PROBABILITIES)
thompson_sampler = ThompsonSampler(n_arms=N_ARMS)

llm_agent = LLMAgent(
    mab,
    model_name=models[0],
    generation_instructions=instructions['vanilla'],
    temperature=LLM_TEMPERATURE)

# --- Run Experiment ---

thompson_dist = run_experiment(mab, thompson_sampler, N, INITIAL_OBSERVATIONS_O)
llm_dist = run_experiment(mab, llm_agent, N, INITIAL_OBSERVATIONS_O)


INFO:rlllm_utils:
            3-armed bandit game. 
            Your goal is to choose the arm that you believe will give you the highest reward.

            A reward of 0 means no success, a reward of 1 means success.
        Here is the history of your previous plays (arm chosen, reward received):
Play 0: Chose Arm 0, Received Reward 0
Play 1: Chose Arm 1, Received Reward 1
Play 2: Chose Arm 0, Received Reward 0
Play 3: Chose Arm 2, Received Reward 1
Play 4: Chose Arm 1, Received Reward 0
Play 5: Chose Arm 2, Received Reward 1
Given the history of actions and rewards, you must choose the next action, an int in [0-n_arms-1]. Note, you should return only the next action, nothing else.
    
INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:google_genai.models:AFC remote call 1 is done.
DEBUG:httpcore.connection:connect_tcp.started host='generativelanguage.googleapis.com' port=443 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect

output 2


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'text/event-stream'), (b'Content-Disposition', b'attachment'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Transfer-Encoding', b'chunked'), (b'Date', b'Thu, 12 Jun 2025 07:32:01 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=2753'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000')])
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-04-17:streamGenerateContent?alt=sse "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
DEBUG:httpcore.connect

output 2


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'text/event-stream'), (b'Content-Disposition', b'attachment'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Transfer-Encoding', b'chunked'), (b'Date', b'Thu, 12 Jun 2025 07:32:03 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=2093'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000')])
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-04-17:streamGenerateContent?alt=sse "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
DEBUG:httpcore.connect

...

output 2


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'text/event-stream'), (b'Content-Disposition', b'attachment'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Transfer-Encoding', b'chunked'), (b'Date', b'Thu, 12 Jun 2025 07:33:10 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=2775'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000')])
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-04-17:streamGenerateContent?alt=sse "HTTP/1.1 200 OK"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete
DEBUG:httpcore.connect

output 2


DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 429, b'Too Many Requests', [(b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Type', b'text/event-stream'), (b'Date', b'Thu, 12 Jun 2025 07:33:11 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'Content-Length', b'1130'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=47'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000')])
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-04-17:streamGenerateContent?alt=sse "HTTP/1.1 429 Too Many Requests"
DEBUG:httpcore.http11:receive_response_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_body.complete
DEBUG:httpcore.http11:response_closed.started
DEBUG:httpcore.http11:response_closed.complete


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'}, 'quotaValue': '10'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '48s'}]}}